In [28]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.neighbors
import sklearn.ensemble
import matplotlib.pyplot as plt
import xgboost
import lightgbm

In [2]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test-full.csv')
train_df.columns

Index(['Id', 'Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
       'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3',
       'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3',
       'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8',
       'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12',
       'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16',
       'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20',
       'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
       'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28',
       'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32',
       'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36',
       'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_

In [3]:
data = train_df.T.to_dict()

In [4]:
train_df_input = train_df.drop(['Id', 'Cover_Type'], axis=1)
train_df_input.values.sum(0)

array([41559587,  2356218,   250338,  3453053,   775833, 25975823,
        3220996,  3309250,  2033294, 23093650,     3568,      569,
           6302,     4681,      339,      627,     1006,      839,
            181,      679,        1,        2,        4,     2096,
            376,      260,      513,      173,        0,      106,
            640,       44,       53,      132,       10,      332,
            742,      265,        6,       48,        8,        7,
           1308,      736,      304,      663,      619,       18,
            103,       14,       32,      744,      634,      456],
      dtype=int64)

In [5]:
test_df_input = test_df.drop(['Id'], axis=1)
test_df_input.values.sum(0)

array([1719426752,   90438473,    8194421,  156541027,   26969912,
       1365463383,  123259400,  129750854,   82810631, 1150572966,
           260796,      29884,     253364,      36968,       3031,
             7525,       4823,      12396,       1597,       6575,
              105,        179,       1147,      32634,      12410,
            29971,      17431,        599,          3,       2845,
             3422,       1899,       4021,       9259,        838,
            33373,      57752,      21278,        474,       2589,
             1086,        946,     115247,      30170,      25666,
            52519,      45154,       1611,       1891,        119,
              298,      15573,      13806,       8750], dtype=int64)

In [6]:
train_df_input = train_df_input.drop(train_df_input.columns[28], axis=1)
test_df_input = test_df_input.drop(train_df_input.columns[28], axis=1)

In [109]:
full_train_X = (train_df_input.values - train_df_input.values.mean(0)) / train_df_input.values.std(0)
full_train_y = train_df['Cover_Type'].values - 1

full_test_X = (test_df_input.values - train_df_input.values.mean(0)) / train_df_input.values.std(0)

label_encoding = True

if label_encoding:
    new_train_X = np.zeros((full_train_X.shape[0], 12))
    new_test_X = np.zeros((full_test_X.shape[0], 12))

    new_train_X[:, :-2] = full_train_X[:, :10]
    new_train_X[:, -2] = np.argmax(full_train_X[:, -43:-39], axis=1)
    new_train_X[:, -1] = np.argmax(full_train_X[:, -39:], axis=1)

    new_test_X[:, :-2] = full_test_X[:, :10]
    new_test_X[:, -2] = np.argmax(full_test_X[:, -43:-39], axis=1)
    new_test_X[:, -1] = np.argmax(full_test_X[:, -39:], axis=1)

    full_test_X = new_test_X
    full_train_X = new_train_X

# pca = sklearn.decomposition.PCA(38)
# pca.fit(full_train_X)
#
# print(pca.explained_variance_ratio_.sum())

# Neighbor-based methods

In [8]:
# scores = sklearn.model_selection.cross_validate(sklearn.neighbors.KNeighborsClassifier(n_neighbors=1), full_train_X, train_df['Cover_Type'], cv=10, scoring='accuracy', return_train_score=True, verbose=2)

# Trees, Boosting

### Random Forest

In [26]:
perform_grid_search = False

if perform_grid_search:
    rf_parameters = {'n_estimators': [80, 100, 150, 300],
                     'max_depth': [6, 10, 20, None],
                     'max_features': ['sqrt', 'log2', 0.5, None],
                     'criterion': ['gini', 'entropy', 'log_loss']}
    grid_search_random_forest = sklearn.model_selection.GridSearchCV(sklearn.ensemble.RandomForestClassifier(),
                                                                     rf_parameters,
                                                                     scoring='accuracy',
                                                                     cv=5,
                                                                     verbose=2)

    grid_search_random_forest.fit(full_train_X, full_train_y)

In [24]:
scores_random_forest = sklearn.model_selection.cross_validate(sklearn.ensemble.RandomForestClassifier(),
                                                              full_train_X,
                                                              full_train_y,
                                                              cv=5,
                                                              verbose=2,
                                                              scoring='accuracy')

scores_random_forest['test_score'].mean()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   1.6s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV] END .................................................... total time=   1.7s
[CV] END .................................................... total time=   1.6s
[CV] END .................................................... total time=   1.5s
[CV] END .................................................... total time=   1.5s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    8.4s finished


0.8595899470899472

### XGBoost

In [ ]:
perform_grid_search = False

if perform_grid_search:
    rf_parameters = {'n_estimators': [80, 100, 150, 300],
                     'max_depth': [6, 10, 20, None],
                     'max_features': ['sqrt', 'log2', 0.5, None],
                     'criterion': ['gini', 'entropy', 'log_loss']}
    grid_search_random_forest = sklearn.model_selection.GridSearchCV(sklearn.ensemble.RandomForestClassifier(),
                                                                     rf_parameters,
                                                                     scoring='accuracy',
                                                                     cv=5,
                                                                     verbose=2)

    grid_search_random_forest.fit(full_train_X, full_train_y)

In [100]:
scores_xgboost = sklearn.model_selection.cross_validate(xgboost.XGBClassifier(verbosity=0, use_label_encoder=False, max_depth=15, eta=0.25, subsample=1.0, colsample_by_tree=1.0),
                                                         full_train_X,
                                                         full_train_y,
                                                         cv=5,
                                                         verbose=2,
                                                         scoring='accuracy')

scores_xgboost['test_score'].mean()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   7.7s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.7s remaining:    0.0s


[CV] END .................................................... total time=   7.6s
[CV] END .................................................... total time=   8.1s
[CV] END .................................................... total time=   7.5s
[CV] END .................................................... total time=   7.5s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   38.7s finished


0.8739417989417989

### LightGBM

In [102]:
perform_grid_search = False

if perform_grid_search:
    lightgbm_parameters = {'max_depth': [15, 20, 25, 30, -1],
                           'colsample_bytree': [0.8, 1.0],
                           'learning_rate': [0.25, 0.3, 0.35, 0.4],
                           'num_leaves': [25, 31, 40, 50, 62],
                           'reg_lambda': [0.0, 1.0],
                           'min_child_samples': [10, 20, 30],
                           'n_estimators': [200]}
    grid_search_lightgbm = sklearn.model_selection.GridSearchCV(lightgbm.sklearn.LGBMClassifier(),
                                                                lightgbm_parameters,
                                                                scoring='accuracy',
                                                                cv=5,
                                                                verbose=2)

    grid_search_lightgbm.fit(full_train_X, full_train_y)

Fitting 5 folds for each of 1200 candidates, totalling 6000 fits
[CV] END colsample_bytree=0.8, learning_rate=0.25, max_depth=15, min_child_samples=10, n_estimators=200, num_leaves=25, reg_lambda=0.0; total time=   1.6s
[CV] END colsample_bytree=0.8, learning_rate=0.25, max_depth=15, min_child_samples=10, n_estimators=200, num_leaves=25, reg_lambda=0.0; total time=   1.9s
[CV] END colsample_bytree=0.8, learning_rate=0.25, max_depth=15, min_child_samples=10, n_estimators=200, num_leaves=25, reg_lambda=0.0; total time=   1.9s
[CV] END colsample_bytree=0.8, learning_rate=0.25, max_depth=15, min_child_samples=10, n_estimators=200, num_leaves=25, reg_lambda=0.0; total time=   1.8s
[CV] END colsample_bytree=0.8, learning_rate=0.25, max_depth=15, min_child_samples=10, n_estimators=200, num_leaves=25, reg_lambda=0.0; total time=   1.8s
[CV] END colsample_bytree=0.8, learning_rate=0.25, max_depth=15, min_child_samples=10, n_estimators=200, num_leaves=25, reg_lambda=1.0; total time=   1.7s
[CV] 

In [126]:
scores_lightgbm = sklearn.model_selection.cross_validate(lightgbm.sklearn.LGBMClassifier(max_depth=20, learning_rate=0.4, n_estimators=150, num_leaves=62, colsample_bytree=1.0, reg_lambda=0.0, min_child_samples=10),
                                                         full_train_X,
                                                         full_train_y,
                                                         cv=5,
                                                         verbose=2,
                                                         scoring='accuracy')

scores_lightgbm['test_score'].mean()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   1.7s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


[CV] END .................................................... total time=   1.8s
[CV] END .................................................... total time=   1.7s
[CV] END .................................................... total time=   1.5s
[CV] END .................................................... total time=   1.5s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    8.6s finished


0.8756613756613756

In [131]:
print(grid_search_lightgbm.best_estimator_)
print(grid_search_lightgbm.best_score_)
grid_search_lightgbm.best_params_

LGBMClassifier(learning_rate=0.25, max_depth=25, min_child_samples=10,
               n_estimators=200, num_leaves=62)
0.8792989417989417


{'colsample_bytree': 1.0,
 'learning_rate': 0.25,
 'max_depth': 25,
 'min_child_samples': 10,
 'n_estimators': 200,
 'num_leaves': 62,
 'reg_lambda': 0.0}

# Make submission

In [127]:
make_submission = True

if make_submission:
    model = lightgbm.sklearn.LGBMClassifier(max_depth=20, learning_rate=0.4, n_estimators=150, num_leaves=62, colsample_bytree=1.0, reg_lambda=0.0, min_child_samples=10)

    test_ids = test_df['Id']

    model.fit(full_train_X, full_train_y)

    test_predictions = model.predict(full_test_X)

    submission_df = pd.DataFrame(data={'Cover_Type': test_predictions + 1})
    submission_df = pd.concat([test_ids, submission_df], axis=1)
    submission_df.to_csv('data/submission.csv', index=False)